In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pickle
from tqdm import tqdm 
from ast import literal_eval
import cobra
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

import gurobipy
gurobipy.setParam('OutputFlag', 0)
import sys
# sys.path.append('../Script/')
from common import *

# input and output

In [2]:
###input
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'
ymdb_uptake_file = '../../Data/ymdb/ymdb_uptake.csv'
spot_result_file = '../../Data/met_spot_result.json'

###output
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'

# add uptake

In [3]:
model = cobra.io.load_yaml_model(yeast8U_del2_path)

In [4]:
model

Name,None
Memory address,7f78f3ba1550
Number of metabolites,3199
Number of reactions,7412
Number of genes,1558
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [5]:
ymdb_uptake = pd.read_csv(ymdb_uptake_file)
print(ymdb_uptake.shape)
ymdb_uptake.head(1)

(51, 5)


,ID,Name,SMILES,Description,inchikey0
0,YMDB02301,Benzoic acid,OC(=O)C1=CC=CC=C1,"Benzoic acid, C6H5COOH, is a colourless crysta...",WPYMKLBDIGXBTP


In [6]:
with open(spot_result_file, 'r', encoding='utf-8') as f:
    met_spot_result = json.load(f)
print(met_spot_result) 

{'YMDB00018': 'YGR186W', 'YMDB00495': 'YEL063C', 'YMDB01639': 'YOR011W', 'YMDB01646': 'YAL034C', 'YMDB01653': 'YPL274W', 'YMDB01658': 'YIL066C', 'YMDB01659': 'YMR295C', 'YMDB01660': 'YNR063W', 'YMDB01661': 'YCR091W', 'YMDB01662': 'YCR091W', 'YMDB01664': 'YIL092W', 'YMDB01665': 'YIL092W', 'YMDB01666': 'YLR192C', 'YMDB01667': 'YCR091W', 'YMDB01679': 'YMR014W', 'YMDB01680': 'YHL040C', 'YMDB01681': 'YCL025C', 'YMDB01682': 'YMR295C', 'YMDB01693': 'YDL132W', 'YMDB01720': 'YOR098C', 'YMDB01721': 'YNL065W', 'YMDB01722': 'YDR011W', 'YMDB01723': 'YIL006W', 'YMDB01724': 'YIL006W', 'YMDB01733': 'YFR034C', 'YMDB01734': 'YLR150W', 'YMDB01735': 'YIL092W', 'YMDB01736': 'YCL073C', 'YMDB01737': 'YCL073C', 'YMDB01738': 'YOL075C', 'YMDB01754': 'YMR295C', 'YMDB01755': 'YCR091W', 'YMDB01756': 'YPL248C', 'YMDB01765': 'YFR050C', 'YMDB01766': 'YIL006W', 'YMDB01767': 'YIL006W', 'YMDB01768': 'YKR106W', 'YMDB01769': 'YFR034C', 'YMDB01770': 'YLR150W', 'YMDB01771': 'YFR016C', 'YMDB01772': 'YCL073C', 'YMDB01774': 'Y

# add YMDB id and YMDB name

In [7]:
metabolites = {}
num = 0
for index, row in ymdb_uptake.iterrows():
    num+=1
    met_id_e = 'snu_'+str(num)
    met_compartment = 'e'
    metabolites[met_id_e] = Metabolite(met_id_e, formula='', name=row['ID'], compartment=met_compartment)

    num+=2
    met_id_c = 'snu_'+str(num)
    met_compartment = 'c'
    metabolites[met_id_c] = Metabolite(met_id_c, formula='', name=row['ID'], compartment=met_compartment)

    reaction = Reaction('rxnu'+str(num))
    reaction.name = 'rxnu'+str(num)
    reaction.subsystem = ''
    reaction.lower_bound = 0.  # This is the default
    reaction.upper_bound = 1000.  # This is the default

    met_e = metabolites.get(met_id_e)
    met_c = metabolites.get(met_id_c)
    reaction.add_metabolites({met_e: -1})
    reaction.add_metabolites({met_c: 1})
    reaction.gene_reaction_rule = met_spot_result[row['ID']]

    try:
        model.reactions.get_by_id('rxnu'+str(num))
    except:
        model.add_reactions([reaction])    

In [8]:
model

Name,None
Memory address,7f78f3ba1550
Number of metabolites,3301
Number of reactions,7463
Number of genes,1582
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [9]:
cobra.io.save_yaml_model(model, yeast8U_del2_path)